# Segmenting and Clustering Neighborhoods in Toronto

First import and install the necessary libraries

In [2]:
# first import some libraries necessary for the actions below
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### Part 1——Scrape table from Wikipedia

Scrape the table with the Toronto postal codes from Wikipedia and put them in a pandas dataframe.

In [ ]:
# get the page from Wikipedia
scrape_url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# use BeautifulSoup to import the table
soup = BeautifulSoup(scrape_url.content,'lxml')
table = soup.find_all('table')[0]
# and put it in a Pandas dataframe
df = pd.read_html(str(table))[0]

In [ ]:
# show first 5 rows of the newly create Data Frame
df.head()

In [ ]:
# check how many instances of "Not Assigned" we have in Borough, and drop those items
df['Borough'].value_counts()

In [ ]:
# We don't need the 'Not assigned' boroughs
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace=True)
# reset the index since we removed rows.
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# check if the Not assigned items are indeed removed
df['Borough'].value_counts()

In [ ]:
# groupby and apply join to have 1 line per post code
df_joined = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [ ]:
df_check1 = df_joined.loc[df_joined['Postcode'] == 'M5A']
df_check2 = df_joined.loc[df_joined['Neighbourhood'] == 'Not assigned']
# check that the Neighbourhoods are indeed combined, with the example from Postcode M5A
print(df_check1)
# check which Neighbourhood is not assigned.
print(df_check2)

In [ ]:
# replace Not assigned with the name of the Borough. 
df_joined['Neighbourhood'].replace('Not assigned', 'Queen\'s Park', inplace=True)
#df_check3 = df_joined.loc[df_joined['Neighbourhood'] == 'Not assigned']
#df_check3
#df_joined.head()

This concludes this part of the exercise.

In [ ]:
df_joined.shape

### Part 2——Create a data frame with FourSquare data

find the latitude and the longitude coordinates of each neighbourhood

In [ ]:
import geocoder # import geocoder

In [ ]:
#g = geocoder.google('Mountain View, CA')
#g.latlng

In [ ]:
# For some reason the geocoder method doesn't work for me. So I decided to use the provided geospatial data.
# first download and import the csv in a data frame.
filename = "https://cocl.us/Geospatial_data"
df_gsdata = pd.read_csv(filename)
df_gsdata.head()

In [ ]:
# rename column Postal Code to Postcode
df_gsdata.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_gsdata.head()

In [ ]:
# merge the two data frames together
df_new = pd.merge(df_joined, df_gsdata, on='Postcode')
df_new.head()

### Part 3——Explore and cluster the neighborhoods in Toronto

Explore and cluster the neighborhoods in Toronto.

In [ ]:
import numpy as np
import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Create a map with the focus on Toronto and plot the neighborhoods from our previous tasks on the map

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(to_latitude, to_longitude))

Plot the locations of the neighborhoods on the Toronto map

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[to_latitude, to_longitude], zoom_start=10)

neighborhoods = df_new
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Zoom in to Down Town Toronto and modify our data frame

In [ ]:
downtown = df_new.loc[df_new['Postcode'] == 'M5A']
# Extract the latitude and the longitude from the data frame
latitude_dt = downtown.iloc[0, 3]
longitude_dt = downtown.iloc[0, 4]

Use the Foursquare API

In [ ]:
search_query = ''
radius = 100
latitude = latitude_dt
longitude = longitude_dt
LIMIT = 100

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url

In [ ]:
results = requests.get(url).json()
#results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.Circle(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [ ]:
dataframe_filtered['categories'].value_counts()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df_new['Neighbourhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#        color=rainbow[cluster-1],
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]